In [1]:
import pandas as pd
import zipfile
import glob

In [2]:
#Read the zip file that contains multiple csv file
from zipfile import ZipFile
from google.colab import files
!wget 'https://raw.githubusercontent.com/Myloveismilk/vn_stock_price/main/files/stock_price_vn.zip'
url = '/content/stock_price_vn.zip'
df = pd.concat(
    [pd.read_csv(ZipFile(url).open(i)) for i in ZipFile(url).namelist()],
    ignore_index=True
)

--2022-10-06 13:10:41--  https://raw.githubusercontent.com/Myloveismilk/vn_stock_price/main/files/stock_price_vn.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 960787 (938K) [application/zip]
Saving to: ‘stock_price_vn.zip’

stock_price_vn.zip  100%[===================>] 938.27K  --.-KB/s    in 0.05s   

2022-10-06 13:10:41 (19.5 MB/s) - ‘stock_price_vn.zip’ saved [960787/960787]



In [3]:
#See the first 5 rows of the dataset
df.head()

,<Ticker>,<DTYYYYMMDD>,<Open>,<High>,<Low>,<Close>,<Volume>
0,BID,20210625,45.1,46.00,44.70,45.55,2572300
1,BID,20210624,45.4,45.90,45.00,45.20,2110700
2,BID,20210623,44.9,46.40,44.90,45.40,2574200
3,BID,20210622,45.3,45.45,44.90,45.00,2781900
4,BID,20210621,45.0,46.00,44.85,45.00,4237600


In [4]:
#The name of the bank is not necessary in this case, so let's remove it
df = df.drop('<Ticker>', axis= 1)

In [5]:
df.columns

Index(['<DTYYYYMMDD>', '<Open>', '<High>', '<Low>', '<Close>', '<Volume>'], dtype='object')

In [6]:
#The names of all columns seem to be not in the right format, so let's change them
df = df.rename(columns= {'<DTYYYYMMDD>':'Date', '<Open>' : 'Open', '<High>': 'High', '<Low>': 'Low', '<Close>': 'Close', '<Volume>':'Volume'})

In [7]:
df.head()

,Date,Open,High,Low,Close,Volume
0,20210625,45.1,46.00,44.70,45.55,2572300
1,20210624,45.4,45.90,45.00,45.20,2110700
2,20210623,44.9,46.40,44.90,45.40,2574200
3,20210622,45.3,45.45,44.90,45.00,2781900
4,20210621,45.0,46.00,44.85,45.00,4237600


In [8]:
#Let's look at the type of each column
df.dtypes

Date        int64
Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object

In [9]:
#We need to change the date format, but first we need to change its data type to object
df['Date'] = df['Date'].astype('object')

In [10]:
#Now let's change the date format
df['Date'] = pd.to_datetime(df['Date'], format= '%Y%m%d')

In [11]:
df.head(10)

,Date,Open,High,Low,Close,Volume
0,2021-06-25,45.10,46.00,44.70,45.55,2572300
1,2021-06-24,45.40,45.90,45.00,45.20,2110700
2,2021-06-23,44.90,46.40,44.90,45.40,2574200
3,2021-06-22,45.30,45.45,44.90,45.00,2781900
4,2021-06-21,45.00,46.00,44.85,45.00,4237600
5,2021-06-18,46.25,46.50,45.70,46.15,3200200
6,2021-06-17,44.60,46.35,44.60,45.90,3579300
7,2021-06-16,44.70,46.70,44.20,45.30,4217600
8,2021-06-15,44.75,45.15,44.00,44.90,3257000
9,2021-06-14,45.50,45.60,44.65,44.90,3394300


In [12]:
# This is a time series project, so the date values have to be in order.
df = df.sort_values('Date')

In [13]:
#The index of each rows is now not in order, let's reset it.
df = df.reset_index(drop= True)

In [14]:
#Change the data type of 'Date' column
df['Date'] = df['Date'].astype('datetime64')

#Set the 'Date' column to index values so it can be easily to plot time series chart.
df = df.set_index('Date')

In [15]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2000-07-28,16.0,16.0,16.0,1.4142,1000
2000-07-31,16.3,16.3,16.3,1.4407,300
2000-08-02,16.6,16.6,16.6,1.4673,100
2000-08-04,16.9,16.9,16.9,1.4938,200
2000-08-07,17.2,17.2,17.2,1.5203,2800


In [16]:
#The dataset now look fine, let's save it for future work.
df.to_csv('stock_price.csv', index = False)